<a href="https://colab.research.google.com/github/PiyushM1/Car-make-model-and-year-classifier/blob/master/Car_make_model_year_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal


### To train three classifiers to predict a car's make, model and year based on inception_v3 model using [Monk](https://github.com/Tessellate-Imaging/monk_v1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import os
os.chdir("/content/drive/My Drive/PilotTask")

# Table of Contents



1.   Install Monk
2.   List item



<a id='0'></a>
# Install Monk
  
 - git clone https://github.com/Tessellate-Imaging/monk_v1.git
 
 - cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt
     - (Select the requirements file as per OS and CUDA version)

In [ ]:
!git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
!cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#!cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#!cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt

## Dataset - Car make, model and year classification
[Cars Train dataset](http://imagenet.stanford.edu/internal/car196/cars_train.tgz)

[Cars Test dataset](http://imagenet.stanford.edu/internal/car196/cars_test.tgz)

In [ ]:
! wget "http://imagenet.stanford.edu/internal/car196/cars_train.tgz" # Train dataset

In [ ]:
!tar -xvf 'cars_train.tgz' -C 'data' #Extracting

In [ ]:
!wget "https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz" # Devkit for the dataset

In [ ]:
!tar -xvf 'car_devkit.tgz' -C 'data' #Extracting

In [ ]:
! wget "http://imagenet.stanford.edu/internal/car196/cars_test.tgz" # Test dataset

In [ ]:
!tar -xvf 'cars_test.tgz' -C 'data' #Extracting

#Loading files containing labels

In [ ]:
from scipy import io
import pandas as pd

For converting the ID to its corresponding label

In [ ]:
class_name = io.loadmat('./data/devkit/cars_meta.mat')

In [ ]:
combined = []
for name in class_name['class_names'][0]:
  name=name.flat[0]
  splits = name.split(" ")
  make = splits[0]
  year = splits[len(splits)-1]
  model = ""
  for tmp in splits[1:len(splits)-2]:
    model = model + tmp + " "
  combined.append([make,model[:-1],year])  

For assigning a label to each image in the train dataset

In [ ]:
train_labels = io.loadmat('./data/devkit/cars_train_annos.mat')

In [ ]:
data=[[row.flat[0] for row in line][4:] for line in train_labels['annotations'][0]]

In [ ]:
# Separating make, model and year for each image and saving them in different csv files
make = [[row[1],combined[row[0]-1][0]] for row in data]
model= [[row[1],combined[row[0]-1][1]] for row in data]
year = [[row[1],combined[row[0]-1][2]] for row in data]

df = pd.DataFrame(make, columns = ['ID', 'Labels'])
df.to_csv("vehicles_make.csv", index = False)
df = pd.DataFrame(model, columns = ['ID', 'Labels'])
df.to_csv("vehicles_model.csv", index = False)
df = pd.DataFrame(year, columns = ['ID', 'Labels'])
df.to_csv("vehicles_year.csv", index = False)

For assigning a label to each image in the test dataset

In [ ]:
# Download test annotations with labels
!wget "http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat" -P './data/devkit'

In [ ]:
test_labels = io.loadmat('./data/devkit/cars_test_annos_withlabels.mat')

In [ ]:
data_test=[[row.flat[0] for row in line][4:] for line in test_labels['annotations'][0]]

In [ ]:
# Separating make, model and year for each image and saving them in different csv files
make = [[row[1],combined[row[0]-1][0]] for row in data_test]
model= [[row[1],combined[row[0]-1][1]] for row in data_test]
year = [[row[1],combined[row[0]-1][2]] for row in data_test]

df = pd.DataFrame(make, columns = ['ID', 'Labels'])
df.to_csv("vehicles_make_test.csv", index = False)
df = pd.DataFrame(model, columns = ['ID', 'Labels'])
df.to_csv("vehicles_model_test.csv", index = False)
df = pd.DataFrame(year, columns = ['ID', 'Labels'])
df.to_csv("vehicles_year_test.csv", index = False)

# Imports

In [77]:
# Monk
import os
import sys
sys.path.append("monk_v1/monk/");

In [78]:
# Installing required libraries
!pip install GPUtil
!pip install pylg

In [79]:
# For Keras backend
from keras_prototype import prototype

# Using mxnet-gluon backend 
# from gluon_prototype import prototype

# For pytorch backend
# from pytorch_prototype import prototype

<a id='3'></a>
# Make Classifier

In [68]:
# Load experiment
gtf = prototype(verbose=1);
gtf.Prototype("Make_classifier", "inception-v3")


gtf.Dataset_Params(dataset_path="data/cars_train",
                   path_to_csv="vehicles_make.csv",
                   split=0.8, input_size=224, 
                   batch_size=64, 
                   shuffle_data=True, 
                   num_processors=3)

# Transform
gtf.apply_random_horizontal_flip(train=True, val=True)

# Set Dataset
gtf.Dataset();

Keras Version: 2.3.1
Tensorflow Version: 2.2.0

Experiment Details
    Project: Make_classifier
    Experiment: inception-v3
    Dir: /content/drive/My Drive/PilotTask/workspace/Make_classifier/inception-v3/

Dataset Details
    Train path:     data/cars_train
    Val path:       None
    CSV train path: vehicles_make.csv
    CSV val path:   None
    Label Type:     single

Dataset Params
    Input Size:   224
    Batch Size:   64
    Data Shuffle: True
    Processors:   3
    Train-val split:   0.8
    Delimiter:   ,

Found 6516 validated image filenames belonging to 49 classes.
Found 1628 validated image filenames belonging to 49 classes.
Pre-Composed Train Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Pre-Composed Val Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Dataset Numbers
    Num train images: 6516
    Num val images:   1628
    Num classes:      49



In [69]:
# Set model parameters
gtf.Model_Params(model_name="inception_v3", freeze_base_network=False, use_gpu=True, use_pretrained=False, gpu_memory_fraction=0.8);

# Load model
gtf.Model()

Model Params
    Model name:           inception_v3
    Use Gpu:              True
    Gpu Memory Fraction:  0.8
    Use pretrained:       False
    Freeze base network:  False

Model Details
    Loading pretrained model
    Model Loaded on device
        Model name:                           inception_v3
        Num layers in model:  190
        Num trainable layers: 190



In [70]:
# Set Training parameters
gtf.Training_Params(num_epochs=10, display_progress=True, display_progress_realtime=True, 
                    save_intermediate_models=True, save_training_logs=True)

# Using RMSprop optimizer
gtf.optimizer_sgd(0.03)

# Fixed learning rate
gtf.lr_fixed()

Training params
    Num Epochs: 10

Display params
    Display progress:          True
    Display progress realtime: True
    Save Training logs:        True
    Save Intermediate models:  True
    Intermediate model prefix: intermediate_model_

Optimizer
    Name:          sgd
    Learning rate: 0.03
    Params:        {'lr': 0.03, 'momentum': 0, 'weight_decay': 0, 'momentum_dampening_rate': 0, 'clipnorm': 0.0, 'clipvalue': 0.0}



Learning rate scheduler
    Name:   fixed
    Params: {}



In [71]:
gtf.loss_crossentropy()

Loss
    Name:          crossentropy
    Params:        {'weight': None, 'batch_axis': 0, 'axis_to_sum_over': -1, 'label_as_categories': True, 'label_smoothing': False}



In [72]:
#Start Training
gtf.Train();

Training Start
Epoch 1/10
101/101 [==============================] - 200s 2s/step - loss: 3.5820 - accuracy: 0.0986 - val_loss: 3.5430 - val_accuracy: 0.1019

Epoch 00001: saving model to workspace/Make_classifier/inception-v3/output/models/resume_state.h5

Epoch 00001: val_loss improved from inf to 3.54301, saving model to workspace/Make_classifier/inception-v3/output/models/best_model.h5

Epoch 00001: saving model to workspace/Make_classifier/inception-v3/output/models/intermediate_model_01.h5
Epoch 2/10
101/101 [==============================] - 175s 2s/step - loss: 3.4716 - accuracy: 0.1086 - val_loss: 3.4799 - val_accuracy: 0.0953

Epoch 00002: saving model to workspace/Make_classifier/inception-v3/output/models/resume_state.h5

Epoch 00002: val_loss improved from 3.54301 to 3.47987, saving model to workspace/Make_classifier/inception-v3/output/models/best_model.h5

Epoch 00002: saving model to workspace/Make_classifier/inception-v3/output/models/intermediate_model_02.h5
Epoch 3/1

<Figure size 432x288 with 0 Axes>

In [ ]:
# Load for validation

gtf = prototype(verbose=1);
gtf.Prototype("Make_classifier", "inception-v3", eval_infer=True);


# Set dataset
gtf.Dataset_Params(dataset_path="data/cars_test",
                   path_to_csv = "vehicles_make_test.csv")
gtf.Dataset();


# Validate
accuracy, class_based_accuracy = gtf.Evaluate();

<a id='11'></a>
# Model Classifier

In [63]:
# Load experiment
gtf = prototype(verbose=1);
gtf.Prototype("Model_classifier", "inception-v3")


gtf.Dataset_Params(dataset_path="data/cars_train",
                   path_to_csv="vehicles_model.csv",
                   split=0.8, 
                   input_size=224, 
                   batch_size=64, 
                   shuffle_data=True,
                   num_processors=3)

# Transform
gtf.apply_random_horizontal_flip(train=True, val=True)

# Set Dataset
gtf.Dataset();

Keras Version: 2.3.1
Tensorflow Version: 2.2.0

Experiment Details
    Project: Model_classifier
    Experiment: inception-v3
    Dir: /content/drive/My Drive/PilotTask/workspace/Model_classifier/inception-v3/

Dataset Details
    Train path:     data/cars_train
    Val path:       None
    CSV train path: vehicles_model.csv
    CSV val path:   None
    Label Type:     multiple

Dataset Params
    Input Size:   224
    Batch Size:   64
    Data Shuffle: True
    Processors:   3
    Train-val split:   0.8
    Delimiter:   ,

Found 6516 validated image filenames belonging to 174 classes.
Found 1628 validated image filenames belonging to 174 classes.
Pre-Composed Train Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Pre-Composed Val Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Dataset Numbers
    Num train images: 6516
    Num val images:   1628
    Num classes:      174



In [64]:
# Set model parameters
gtf.Model_Params(model_name="inception_v3", freeze_base_network=False, use_gpu=True, use_pretrained=False, gpu_memory_fraction=0.8);

# Load model
gtf.Model()

Model Params
    Model name:           inception_v3
    Use Gpu:              True
    Gpu Memory Fraction:  0.8
    Use pretrained:       False
    Freeze base network:  False

Model Details
    Loading pretrained model
    Model Loaded on device
        Model name:                           inception_v3
        Num layers in model:  190
        Num trainable layers: 190



In [65]:
# Set Training parameters
gtf.Training_Params(num_epochs=5, display_progress=True, display_progress_realtime=True, 
                    save_intermediate_models=True, save_training_logs=True)

# Using RMSprop optimizer
gtf.optimizer_sgd(0.03)

# Fixed learning rate
gtf.lr_fixed()

Training params
    Num Epochs: 5

Display params
    Display progress:          True
    Display progress realtime: True
    Save Training logs:        True
    Save Intermediate models:  True
    Intermediate model prefix: intermediate_model_

Optimizer
    Name:          sgd
    Learning rate: 0.03
    Params:        {'lr': 0.03, 'momentum': 0, 'weight_decay': 0, 'momentum_dampening_rate': 0, 'clipnorm': 0.0, 'clipvalue': 0.0}



Learning rate scheduler
    Name:   fixed
    Params: {}



In [66]:
gtf.loss_crossentropy()

Loss
    Name:          crossentropy
    Params:        {'weight': None, 'batch_axis': 0, 'axis_to_sum_over': -1, 'label_as_categories': True, 'label_smoothing': False}



In [67]:
#Start Training
gtf.Train();

Training Start
Epoch 1/5
101/101 [==============================] - 199s 2s/step - loss: 5.1564 - accuracy: 0.0074 - val_loss: 5.1545 - val_accuracy: 0.0088

Epoch 00001: saving model to workspace/Model_classifier/inception-v3/output/models/resume_state.h5

Epoch 00001: val_loss improved from inf to 5.15450, saving model to workspace/Model_classifier/inception-v3/output/models/best_model.h5

Epoch 00001: saving model to workspace/Model_classifier/inception-v3/output/models/intermediate_model_01.h5
Epoch 2/5
101/101 [==============================] - 174s 2s/step - loss: 5.1150 - accuracy: 0.0107 - val_loss: 5.0822 - val_accuracy: 0.0102

Epoch 00002: saving model to workspace/Model_classifier/inception-v3/output/models/resume_state.h5

Epoch 00002: val_loss improved from 5.15450 to 5.08221, saving model to workspace/Model_classifier/inception-v3/output/models/best_model.h5

Epoch 00002: saving model to workspace/Model_classifier/inception-v3/output/models/intermediate_model_02.h5
Epoch

<Figure size 432x288 with 0 Axes>

In [ ]:
# Load for validation

gtf = prototype(verbose=1);
gtf.Prototype("Model_classifier", "resnet50-v2", eval_infer=True);


# Set dataset
gtf.Dataset_Params(dataset_path="data/cars_test",
                   path_to_csv = "vehicles_model_test.csv")
gtf.Dataset();


# Validate
accuracy, class_based_accuracy = gtf.Evaluate();

#Year Classifier

In [49]:
# Load experiment
gtf = prototype(verbose=1);
gtf.Prototype("Year_classifier", "inception-v3")


gtf.Dataset_Params(dataset_path="data/cars_train",
                   path_to_csv="vehicles_year.csv",
                   split=0.8, input_size=224, 
                   batch_size=64,
                   shuffle_data=True, 
                   num_processors=3)

# Transform
gtf.apply_random_horizontal_flip(train=True, val=True)

# Set Dataset
gtf.Dataset();

Keras Version: 2.3.1
Tensorflow Version: 2.2.0

Experiment Details
    Project: Year_classifier
    Experiment: inception-v3
    Dir: /content/drive/My Drive/PilotTask/workspace/Year_classifier/inception-v3/

Dataset Details
    Train path:     data/cars_train
    Val path:       None
    CSV train path: vehicles_year.csv
    CSV val path:   None
    Label Type:     single

Dataset Params
    Input Size:   224
    Batch Size:   64
    Data Shuffle: True
    Processors:   3
    Train-val split:   0.8
    Delimiter:   ,

Found 6516 validated image filenames belonging to 16 classes.
Found 1628 validated image filenames belonging to 16 classes.
Pre-Composed Train Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Pre-Composed Val Transforms
[{'RandomHorizontalFlip': {'p': 0.5}}]

Dataset Numbers
    Num train images: 6516
    Num val images:   1628
    Num classes:      16



In [50]:
# Set model parameters
gtf.Model_Params(model_name="inception_v3", freeze_base_network=False, use_gpu=True, use_pretrained=False, gpu_memory_fraction=0.8);

# Load model
gtf.Model()

Model Params
    Model name:           inception_v3
    Use Gpu:              True
    Gpu Memory Fraction:  0.8
    Use pretrained:       False
    Freeze base network:  False

Model Details
    Loading pretrained model
    Model Loaded on device
        Model name:                           inception_v3
        Num layers in model:  190
        Num trainable layers: 190



In [51]:
# Set Training parameters
gtf.Training_Params(num_epochs=5, display_progress=True, display_progress_realtime=True, 
                    save_intermediate_models=True, save_training_logs=True)

# Using RMSprop optimizer
gtf.optimizer_sgd(0.01)

# Fixed learning rate
gtf.lr_fixed()

Training params
    Num Epochs: 5

Display params
    Display progress:          True
    Display progress realtime: True
    Save Training logs:        True
    Save Intermediate models:  True
    Intermediate model prefix: intermediate_model_

Optimizer
    Name:          sgd
    Learning rate: 0.01
    Params:        {'lr': 0.01, 'momentum': 0, 'weight_decay': 0, 'momentum_dampening_rate': 0, 'clipnorm': 0.0, 'clipvalue': 0.0}



Learning rate scheduler
    Name:   fixed
    Params: {}



In [52]:
gtf.loss_crossentropy()

Loss
    Name:          crossentropy
    Params:        {'weight': None, 'batch_axis': 0, 'axis_to_sum_over': -1, 'label_as_categories': True, 'label_smoothing': False}



In [53]:
#Start Training
gtf.Train();

Training Start
Epoch 1/5
101/101 [==============================] - 210s 2s/step - loss: 1.7113 - accuracy: 0.5778 - val_loss: 1.8276 - val_accuracy: 0.5938

Epoch 00001: saving model to workspace/Year_classifier/inception-v3/output/models/resume_state.h5

Epoch 00001: val_loss improved from inf to 1.82757, saving model to workspace/Year_classifier/inception-v3/output/models/best_model.h5

Epoch 00001: saving model to workspace/Year_classifier/inception-v3/output/models/intermediate_model_01.h5
Epoch 2/5
101/101 [==============================] - 175s 2s/step - loss: 1.6092 - accuracy: 0.5879 - val_loss: 1.6737 - val_accuracy: 0.5953

Epoch 00002: saving model to workspace/Year_classifier/inception-v3/output/models/resume_state.h5

Epoch 00002: val_loss improved from 1.82757 to 1.67365, saving model to workspace/Year_classifier/inception-v3/output/models/best_model.h5

Epoch 00002: saving model to workspace/Year_classifier/inception-v3/output/models/intermediate_model_02.h5
Epoch 3/5
1

<Figure size 432x288 with 0 Axes>

In [ ]:
# Load for validation

gtf = prototype(verbose=1);
gtf.Prototype("Year_classifier", "inception_v3", eval_infer=True);


# Set dataset
gtf.Dataset_Params(dataset_path="data/cars_test",
                   path_to_csv = "vehicles_year_test.csv")
gtf.Dataset();


# Validate
accuracy, class_based_accuracy = gtf.Evaluate();